<a href="https://colab.research.google.com/github/ElleStark/5343FinalProject/blob/main/FTLE_computation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FTLE Fields - Computation Tutorial

This notebook contains procedures and example Python code for computing and plotting finite-time Lyapunov exponent (FTLE) fields from velocity field data.

### Background

FTLE fields are useful as a diagnostic method for identifying Lagrangian coherent structures (LCS) in fluid flows. LCS are material surfaces that organize a fluid flow into ordered patterns that frame and quantify material transport, forming what's been described as a robust 'skeleton' of the flow (Haller 2015). Ridges in backward-time FTLE fields indicate the presence of attracting LCS, which, as the name implies, attract nearby fluid trajectories at the highest local rate, while ridges in forward-time FTLE fields indicate the presence of repelling LCS, which repel nearby fluid trajectories at the highest local rate (Peacock and Haller 2013). In our example code, we develop a backward-time FTLE field for a double gyre flow with an analytical velocity field.

### Overview

The tutorial is organized into 7 steps:

1.   Define the velocity field
2.   Advect a grid of particles to determine the flow map $ϕ$
3.   Compute the gradient of the flow map using finite differencing: \\
$\frac{d \phi_t^{t+T}(\bf x)}{d \bf x} \bigg |_{\mathbf{x}_{i,j}} = \begin{bmatrix}
\frac{x_{i+1,j}(t+T) - x_{i-1,j}(t+T)}{x_{i+1,j}(t) - x_{i-1,j}(t)} & \frac{x_{i,j+1}(t+T) - x_{i,j-1}(t+T)}{y_{i,j+1}(t) - y_{i,j-1}(t)} \\
\frac{y_{i+1,j}(t+T) - y_{i-1,j}(t+T)}{x_{i+1,j}(t) - x_{i-1,j}(t)} & \frac{y_{i,j+1}(t+T) - y_{i,j-1}(t+T)}{y_{i,j+1}(t) - y_{i,j-1}(t)}
\end{bmatrix}$
4.   Calculate the Cauchy strain tensor, given by: \\
$\Delta = \frac{d \phi_t^{t+T}(\bf x)}{d \bf x}^*\frac{d \phi_t^{t+T}(\bf x)}{d \bf x}$
5.   Find the maximum eigenvalue of the Cauchy strain tensor, $λ_{max}$
6.   Compute the FTLE field, given by: \\
$\sigma_t^T(\mathbf x) = \frac{1}{|T|}\ln \sqrt{λ_{max}(Δ)}$
7.   Plot the FTLE field

The procedures below generally follow the approach described in Shadden et al., 2005, and draws from the Shadden lab tutorial for LCS computation: https://shaddenlab.berkeley.edu/uploads/LCS-tutorial/computation.html.  

## 1. Define the velocity field

Use a callable function to define the instantaneous velocity at a given time at one or more points in space. For an analytical flow, the function can take arguments including time and location(s), which are plugged into the velocity equation(s) to calculate the velocity at any point in time and space, as in our example flow. For a discrete flow, the function would estimate the velocity at a desired point based on interpolating from points with known/measured velocity.

\\

The example function below calculates the velocity field for a double gyre, which models the incrompressible flow of two counter-rotating vortices that periodically expand and contract. The velocity field is given by:

$u = -\pi U \sin (\pi f(x)) \cos(\pi y)$, \\
$v = \pi U \cos(\pi f(x)) \sin(\pi y) \frac{∂ f}{∂ x}$, \\
where \\
$f(x, t) = x[1 + ϵ \sin(2 \pi t/T_0)(x-2)]$. \\

In the example below we set $ϵ = 0.25$, $T_0=10$, and $a=0.1$, following Pratt et al., 2015. \\



First, we need to import all required Python libraries:

In [ ]:
import numpy as np
import numpy.linalg as LA
from math import *
import matplotlib.pyplot as plt
import matplotlib.animation as animation

Then, we can define a function to calculate (or interpolate, for discrete fields) the velocity field at a given point(s) in space and time:

In [ ]:
def vfield(time, y, epsilon=0.25, T_0=10, a=0.1):

  f = y[0] * [1 + epsilon * np.sin(2 * pi * time / T_0) * (y[0] - 2)]
  df = 1 + 2 * epsilon * (y[0] - 1) * np.sin(2 * pi * time / T_0)

  u = -pi * a * np.sin(pi * f) * np.cos(pi * y[1])
  v = pi * a * np.cos(pi * f) * np.sin(pi * y[1]) * df
  u = np.squeeze(u)  # get rid of extra dimension of length 1 if present
  v = np.squeeze(v)

  vfield = np.array([u, v])  # convert to array for vectorized FTLE calculations later

  return vfield

To check our velocity function, we can calculate and plot the field at a few times:

In [ ]:
# Initialize variables
num_x_points = 40
T_0 = 10
plot_times = [0, 0.25*T_0, 0.75*T_0]
vfields = []

# Create small grid for showing velocity arrows
x = np.linspace(0, 2, num_x_points)
y = np.linspace(0, 1, int(num_x_points/2))
x, y = np.meshgrid(x, y, indexing='xy') # set grid indexing to xy for Cartesian indexing

# Loop through time, assigning velocity field [x, y, u, v] for each t
for time in plot_times:
  velocity_field = vfield(time, [x, y])
  # need to extract u and v from vfield array
  u = velocity_field[0]
  v = velocity_field[1]
  velocity_field = [x, y, u, v]
  vfields.append(velocity_field)

# Store vfields for all times in dictionary
vfield_dict = dict(zip(plot_times, vfields))

# Plot the velocity field, one image for each time
for time in plot_times:
   plt.quiver(*vfield_dict[time])
   plt.show()

## 2. Advect a grid of particles to determine the flow map
First initialize a grid of particles, then advect them over the integration time using the velocity data from step 2. For ***attracting LCS*** boundaries, advect the particles ***backward in time***. For ***repelling LCS*** boundaries, advect the particles ***forward in time***. To advect the particles, a ***Runge-Kutta 4th-order integration*** (RK4) algorithm is often used, though lower-order integration schemes could also be used if computation time is an issue.

\\

Note that the integration time plays a key role in the resulting FTLE field and will be dependent on the application. Generally, longer integration time results in more refined and lengthened LCS. Our example uses an integration time of 2 periods, based on the selection in Pratt et al., 2015, to resolve the dynamics of the double gyre. For additional discussion of integration time, see Shadden et al., 2005.

For additional details about RK4 implementation in Python, see these videos by Steve Brunton: \\

Coding RK4 in Python & Matlab - https://www.youtube.com/watch?v=vNoFdtcPFdk \\
Vectorized integration for bundle of initial conditions - https://www.youtube.com/watch?v=LRF4dGP4xeo  

\\

First, we define functions to 1) implement RK4 to advect the particles a single timestep and 2) compute the final particle positions at the end of the total integration time.  

In [ ]:
### Function to estimate particle position after one timestep using 4th order Runge-Kutta method
def rk4_singlestep(vfield, dt, t0, y0):

  # Compute velocity at full steps and partial steps
  f1 = vfield(t0, y0)
  f2 = vfield(t0 + dt / 2, y0 + (dt / 2) * f1)
  f3 = vfield(t0 + dt / 2, y0 + (dt / 2) * f2)
  f4 = vfield(t0 + dt, y0 + dt * f3)
  # Take a weighted average * dt to move the particle
  y_out = y0 + (dt / 6) * (f1 + 2 * f2 + 2 * f3 + f4)
  return y_out


### Function to estimate flow map (final particle positions) using RK4
def compute_flow_map(x, y, T, start_time):

  # Initialize variables
  dt = T / 1000 # Here timestep is defined based on the integration time
  L = abs(int(T / dt))
  ny, nx = np.shape(x)

  # Se up Initial Conditions
  yIC = np.zeros((2, nx * ny))
  yIC[0, :] = x.reshape(nx * ny)
  yIC[1, :] = y.reshape(nx * ny)
  yin = yIC

  # Compute new positions at each step
  for step in range(L):
      tstep = step * dt + start_time
      yout = rk4_singlestep(vfield, dt, tstep, yin)
      yin = yout

  return yout

Once we've defined our functions, we can call them to compute a flow map for a specified flow and integration time. The example below advects a grid of 200 x 400 particles backward in time 2 periods using the double gyre velocity field defined in step 1.

In [ ]:
# Set up grid of particles
num_x_points = 400
x = np.linspace(0, 2, num_x_points)
y = np.linspace(0, 1, int(num_x_points/2))
x, y = np.meshgrid(x, y, indexing='xy') # set grid indexing to xy for Cartesian indexing

# Assign variables: period T_0 and integration time T
T_0 = 10
T = -2 * T_0
start_time = 0

# Call function to compute flow map
flow_map = compute_flow_map(x, y, T, start_time)

To check that the particles moved, we can plot the original and final particle locations for a small section of the grid:

In [ ]:
# Original grid
fig, ax = plt.subplots()
plt.scatter(x, y)
ax.set(xlim=(0, 0.1), ylim=(0, 0.05)) # Zoom in to see individual particles
plt.show()

# Final grid
fig, ax = plt.subplots()
plt.scatter(flow_map[0], flow_map[1])
ax.set(xlim=(0, 0.1), ylim=(0, 0.05)) # Zoom in to see individual particles
plt.show()

## 3. Compute the gradient of the flow map

In Step 2, we obtained a map of particle locations from initial positions $\mathbf{x}_{i, j}(t)$ to final positions $\mathbf{x}_{i, j}(t+T)$. As long as our initial particle locations are close enough to each other, we can calculate the gradient of the flow map $\phi$ using central differencing with the following formula (Shadden et al., 2005):

$\frac{d \phi_t^{t+T}(\bf x)}{d \bf x} \bigg |_{\mathbf{x}_{i,j}} = \begin{bmatrix}
\frac{x_{i+1,j}(t+T) - x_{i-1,j}(t+T)}{x_{i+1,j}(t) - x_{i-1,j}(t)} & \frac{x_{i,j+1}(t+T) - x_{i,j-1}(t+T)}{y_{i,j+1}(t) - y_{i,j-1}(t)} \\
\frac{y_{i+1,j}(t+T) - y_{i-1,j}(t+T)}{x_{i+1,j}(t) - x_{i-1,j}(t)} & \frac{y_{i,j+1}(t+T) - y_{i,j-1}(t+T)}{y_{i,j+1}(t) - y_{i,j-1}(t)}
\end{bmatrix}$



In [ ]:
# Assign and initialize variables
grid_height, grid_width = np.shape(x)
x_final = flow_map[0]
x_final = x_final.reshape(grid_height, grid_width)
y_final = flow_map[1]
y_final = y_final.reshape(grid_height, grid_width)

delta_x = x[0][0] - x[0][1]
delta_y = y[0][0] - y[1][0]

jacobian = np.empty([grid_height, grid_width, 2, 2], float)

# Loop through each point, calculating the gradient (Jacobian) at each location
for i in range(1, grid_width - 1):
    for j in range(1, grid_height - 1):
        jacobian[j][i][0][0] = (x_final[j, i + 1] - x_final[j, i - 1]) / (2 * delta_x)
        jacobian[j][i][0][1] = (x_final[j + 1, i] - x_final[j - 1, i]) / (2 * delta_y)
        jacobian[j][i][1][0] = (y_final[j, i + 1] - y_final[j, i - 1]) / (2 * delta_x)
        jacobian[j][i][1][1] = (y_final[j + 1, i] - y_final[j - 1, i]) / (2 * delta_y)

## 4. Calculate the Cauchy strain tensor

The Cauchy strain tensor, $Δ$, (aka the right Cauchy-Green deformation tensor) provides a rotation-independent description of the square of local change in distances due to deformation. We compute it by multiplying the deformation gradient tensor by its transpose as follows:

$\Delta = \frac{d \phi_t^{t+T}(\bf x)}{d \bf x}^*\frac{d \phi_t^{t+T}(\bf x)}{d \bf x}$

In [ ]:
cg_tensor = np.empty([grid_height, grid_width, 2, 2], float)

for i in range(1, grid_width - 1):
    for j in range(1, grid_height - 1):
      cg_tensor[j][i] = np.dot(np.transpose(jacobian[j][i]), jacobian[j][i])

## 5. Find the largest eigenvalue of the deformation tensor

We then compute the largest eigenvalue, $λ_{max}$, of the 2 x 2 Cauchy strain tensor at each location.  

In [ ]:
max_eigvals = np.empty([grid_height, grid_width])

for i in range(1, grid_width - 1):
    for j in range(1, grid_height - 1):
      eigvals = LA.eigvals(cg_tensor[j][i])
      max_eigvals[j][i] = max(eigvals)

## 6. Compute FTLE values

The max eigenvalue can now be plugged into the below equation to compute the FTLE field, $σ_t^T(\mathbf{x})$, at each position in the particle grid.

$\sigma_t^T(\mathbf x) = \frac{1}{|T|}\ln \sqrt{λ_{max}(Δ)}$

In [ ]:
ftle = np.zeros([grid_height, grid_width], float)

for i in range(1, grid_width - 1):
    for j in range(1, grid_height - 1):
      ftle[j][i] = 1 / (2 * abs(T)) * log(sqrt(max_eigvals[j][i]))

## 7. Plot FTLE field

We can now visualize LCS boundaries by plotting the FTLE field. Ridges in the backward-time FTLE fields represent attracting LCS, while ridges in forward-time FTLE fields represent repelling LCS. We expect passive scalars to coalesce along attracting LCS.

In [ ]:
# Contour plot of backward-time FTLE field calculated above
fig, ax = plt.subplots()
plt.contourf(x, y, ftle, 100, cmap=plt.cm.Greys)
ax.set_aspect('equal', adjustable='box')

## Extensions

This notebook provided an example of calculating an FTLE field at one point in time for an analytical velocity field. There are many potentially useful extensions to the above methods, such as computing FTLE fields for many points in time to visualize the dynamic behavior of the flow or computing FTLE fields based on discrete/experimental data. An expanded version of the above code that includes FTLE field animations is available here: https://github.com/ElleStark/5343FinalProject.   

## References
Haller, G. (2015). Lagrangian Coherent Structures. Annual Review of Fluid Mechanics, 47, 137-62. https://doi.org/10.1146/annurev-fluid-010313-141322

\\

Peacock, T., & Haller, G. (2013). Lagrangian coherent structures: The hidden skeleton of fluid flows. Physics Today, 66(2), 41–47. https://doi.org/10.1063/PT.3.1886

\\

Pratt, K. R., Meiss, J. D., & Crimaldi, J. P. (2015). Reaction enhancement of initially distant scalars by Lagrangian coherent structures. Physics of Fluids, 27(3), 035106. https://doi.org/10.1063/1.4914467

\\

Shadden, S. C., Lekien, F., & Marsden, J. E. (2005). Definition and properties of Lagrangian coherent structures from finite-time Lyapunov exponents in two-dimensional aperiodic flows. Physica D: Nonlinear Phenomena, 212(3–4), 271–304. https://doi.org/10.1016/j.physd.2005.10.007

